In [ ]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
import ipywidgets as widgets
from IPython.display import display
#### conectar_base_de_Datos
#!pip install ipywidgets

# CREAR CONEXIÓN CON LA BASE DE DATOS db_movies
con = sql.connect('data/db_movies')

# CREAR EL CURSOR
cur = con.cursor() ## se crea el cursor, que es el otro tipo de conexión para ejecutar las consultas

In [ ]:
# Cargar todos los datos una sola vez desde full_ratings
df_full = pd.read_sql('SELECT * FROM full_ratings', con)

# Seleccionar las columnas que que influiran en el modelo de vecinos cercanos
feature_cols = df_full.drop(columns=['movie_id', 'movie_title', 'user_id']).columns

# Detectar automáticamente todas las columnas dummy de géneros (suponiendo que no son 'movie_id', 'user_id', etc.)
columnas_excluir = {'movie_id', 'movie_title', 'user_id', 'movie_rating','movie_title','movie_year'}  # agrega más si es necesario
genre_cols = [col for col in df_full.columns if col not in columnas_excluir and df_full[col].dropna().isin([0, 1]).all()]

# Dropdown interactivo con todos los géneros detectados
dropdown_genero = widgets.Dropdown(
    options=genre_cols,
    description='Género:',
    layout=widgets.Layout(width='300px')
)

def recomendar_con_genero(user_id, genero, n_recomendaciones=10):
    ratings_user = df_full[df_full['user_id'] == user_id]
    #if ratings_user.empty:
    #    return pd.DataFrame({'mensaje': [f' Usuario {user_id} sin registros.']})

    rated_ids = ratings_user['movie_id'].unique()
    df_rated = ratings_user[feature_cols].copy()
    df_rated['dummy'] = 1
    perfil = df_rated.groupby('dummy').mean() 

    # Filtrar solo películas no vistas y que pertenezcan al género seleccionado
    df_no_rated = df_full[(~df_full['movie_id'].isin(rated_ids)) & (df_full[genero] == 1)]
    df_no_rated = df_no_rated.drop_duplicates('movie_id')
    X_no_rated = df_no_rated[feature_cols]

    #if X_no_rated.empty:
    #    return pd.DataFrame({'mensaje': [f' No hay películas del género {genero} para recomendar.']})

    model = neighbors.NearestNeighbors(n_neighbors=n_recomendaciones, metric='cosine')
    model.fit(X_no_rated)
    dist, idx = model.kneighbors(perfil)  # <- Línea corregida

    recs = df_no_rated.iloc[idx[0]][['movie_title', 'movie_id']].copy()
    recs['distancia'] = dist[0]
    return recs.sort_values(by='distancia')

def mostrar_recomendaciones(genero):
    display(recomendar_con_genero(user_id=609, genero=genero))

# Crear el widget interactivo
widgets.interact(mostrar_recomendaciones, genero=dropdown_genero)

interactive(children=(Dropdown(description='Género:', layout=Layout(width='300px'), options=('Action', 'Advent…

<function __main__.mostrar_recomendaciones(genero)>